# Ejercicio: clasificando tweets con modelos Transformers

<img src="https://albarji-labs-materials.s3.eu-west-1.amazonaws.com/transformers.jpg">
<div align="right"><a href=http://wallur.com/wallpaper/36471>Image source</a></div>

En este notebook vamos a abordar la tarea de clasificar pequeños documentos de texto (tweets), a base de hacer un ajuste fine de modelos Transformer pre-entrenados.

Se recomienda que ejecutes este notebook en [Google Colaboratory](https://colab.research.google.com/). Asegúrate de [activar el uso de la GPU](https://colab.research.google.com/notebooks/gpu.ipynb).

## Preliminares

Primero instalaremos Transformers y otras librerías relacionadas.

In [1]:
pip install transformers==4.18.* datasets==2.0.*

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 325 kB 37.1 MB/s 
     |████████████████████████████████| 6.6 MB 30.9 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 34.8 MB/s 
     |████████████████████████████████| 895 kB 35.3 MB/s 
     |████████████████████████████████| 212 kB 48.4 MB/s 
     |████████████████████████████████| 1.1 MB 41.5 MB/s 
     |████████████████████████████████| 136 kB 49.4 MB/s 
     |████████████████████████████████| 127 kB 50.4 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |████████████████████████████████| 271 kB 41.6 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

Vamos a comprobar que esta instancia tiene una GPU conectada. El siguiente comando debería devolver información sobre el modelo de GPU.

In [2]:
!nvidia-smi

Sat Apr 16 10:41:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Carga de datos

Para este ejercicio utilizaremos [tweet_eval](https://huggingface.co/datasets/tweet_eval), un dataset de tweets ya preparado y disponible en el hub de Huggingface. Para cargarlo en este notebook podemos hacer uso de la librería [Datasets](https://huggingface.co/docs/datasets/index). Dado que este dataset incluye varios objetivos de clasificación, vamos a seleccionar el de clasificación de emociones (`"emotion"`).

In [3]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emotion")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Si analizamos la estructura de este objeto `Dataset`, veremos que se compone de particiones de `train`, `validation` y `test`. Cada una de ellas contiene tanto los textos de cada tweet (`text`) como su clase asignada (`label`).

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3257
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1421
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 374
    })
})

Podemos ver los contenidos del primer texto de `train` de la siguiente forma.

In [5]:
dataset["train"][0]

{'label': 2,
 'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry"}

Los objetos `Dataset` permiten también hacer slicing, pudiendo así seleccionar un subconjunto de los datos. Por ejemplo, podemos tomar los 10 primeros datos de `train` de la siguiente manera.

In [6]:
dataset["train"][0:10]

{'label': [2, 0, 1, 0, 3, 0, 3, 1, 0, 0],
 'text': ["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
  "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs",
  "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu",
  "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight",
  "it's pretty depressing when u hit pan on ur favourite highlighter",
  '@user but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .',
  'Making that yearly transition from excited and hopeful college returner to sick and exhausted pessimist. #college',
  'Tiller and breezy should do a collab album. Rapping and singing prolly be fire',
  '@user broadband is shocking regretting signing up now #angry #shouldofgonewithvirgin',
  '@user Look at those teef! #growl']}

Una información relevante que no se incluye en el objeto `Dataset` es el significado de cada etiqueta de clasificación. Para ello, podemos consultar la [descripción de este dataset en el hub](https://huggingface.co/datasets/tweet_eval). Por comodidad, vamos a crear aquí un diccionario con la correspondencia entre cada índice de clase y su nombre.

In [7]:
class_labels = {
    0: "anger",
    1: "joy",
    2: "optimism",
    3: "sadness"
}

## Tokenización

El primer paso en cualquier modelo de lenguaje es tokenizar los textos. Podemos hacer esto utilizando un `AutoTokenizer` de la librería [Transformers](https://huggingface.co/docs/transformers/index).

In [8]:
from transformers import AutoTokenizer

Los tokenizadores son elementos de procesamiento que están íntimamente ligados con el modelo de lenguaje que vamos a utilizar. Para este ejercicio vamos a emplear el modelo [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased), el cual es un modelo BERT comprimido que permite obtener un rendimiento razonable con poco coste computacional.

Para instanciar el tokenizador correspondiente a DistilBERT, podemos hacerlo de la siguiente manera

In [9]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Podemos comprobar cómo funciona el tokenizador con un pequeño texto

In [10]:
tokenizer.tokenize("A long trip to Mordor")

['a', 'long', 'trip', 'to', 'mor', '##dor']

La mayoría de modelos de lenguaje utilizan alguna estrategia para tokenizar el texto en trozos de palabras, especialmente cuando se trata de palabras poco frecuentes en el idioma. Esta estrategia permite generalizar a palabras nunca vistas. En este caso podemos comprobar cómo la palabra "Mordor" se ha dividido en dos trozos de palabra: "mor" y "##dor". Los marcadores "##" indican que ese trozo de palabra es continuación de una palabra anterior.

Es también importante remarcar que estamos usando un modelo de lenguaje de tipo "uncased", lo que significa que no tendrá en cuenta las diferencias entre mayúsculas y minúsculas. Debido a esto, el tokenizador transforma todos los textos a minúsculas.

También podemos utilizamos el objeto tokenizador como si fuera una función, en cuyo caso nos devolverá una codificación del texto apta para el entrenamiento de modelos:

In [11]:
encoded_text = tokenizer("A long trip to Mordor")
encoded_text

{'input_ids': [101, 1037, 2146, 4440, 2000, 22822, 7983, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Esta codificación nos devuelve un diccionario con varios elementos:
* `input_ids`: identificadores de los tokens extraídos del texto.
* `attention_mask`: indicadores 0/1 marcando si las capas de atención del modelo tienen que considerar ese token en sus cálculos o no. En este ejemplo se emplean todos los tokens (`1`), pero más adelante veremos casos en los que se generan tokens que pueden ignorarse.
* (opcionalmente) `token_types_ids`: para modelos de lenguaje que aprenden sobre pares de frases, este campo contiene indicadores 0/1 marcando a qué frase corresponde el token. Este no es el caso del modelo DistilBERT, por lo que este atributo no existe.

Nótese que esta codificación también nos añade los tokens especiales `[CLS]` and `[SEP]`, los cuales son necesarios en los modelos tipo BERT para marcar el inicio y el fin de cada texto. Podemos evidenciar esto traduciendo esta codificación de vuelta a trozos de palabras.

In [12]:
for token_id in encoded_text["input_ids"]:
    print(f'{token_id} -> {tokenizer.decode([token_id])}')

101 -> [CLS]
1037 -> a
2146 -> long
4440 -> trip
2000 -> to
22822 -> mor
7983 -> ##dor
102 -> [SEP]


Visto el funcionamiento básico del tokenizador, podemos utilizarlo ahora para tokenizar todo nuestro dataset. Esto podemos hacerlo de forma muy conveniente con la función `map` del dataset, que nos permite ejecutar una función sobre cada dato y guardar los resultados como nuevos atributos del dataset. En este caso, la función que vamos a aplicar es llamar al tokenizador con cada texto, configurando la tokenización de la siguiente manera:

* `max_length=60`: solo se utilizarán 60 tokens de cada texto.
* `padding='max_length'`: si algún texto tiene menos de 60 tokens, se le añadirán unos tokens especiales `[PAD]` hasta llegar a 60 tokens.
* `truncation=True`: si algún texto tiene más de 60 tokens, se descartarán tokens de su final hasta quedar en 60.

El entrenamiento del modelo va a requerir que todos los textos tengan el mismo número de tokens, por lo que esta configuración los encaja todos a 60 tokens. Este número de tokens será suficiente para este dataset de tweets, pero en otros problemas con textos más largos es posible que necesitemos un mayor número de tokens para obtener un mejor rendimiento.

Nótese también que se usa la opción `batched=True` para que la función `map` realice la tokenización en batches, lo cual resulta en un procesamiento más eficiente.

In [13]:
tokenized_dataset = dataset.map(
    lambda example: tokenizer(example["text"], max_length=60, padding='max_length', truncation=True),
    batched=True
)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Podemos comprobar ahora qué aspecto tiene el dataset tokenizado

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3257
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1421
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 374
    })
})

La tokenización ha añadido a cada texto los atributos, `input_ids` y `attention_mask`, tal y como esperábamos. Podemos analizar un texto en concreto para comprobar cómo ha quedado tras la tokenización.

In [15]:
print(tokenized_dataset["train"][0])

{'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry", 'label': 2, 'input_ids': [101, 1523, 4737, 2003, 1037, 2091, 7909, 2006, 1037, 3291, 2017, 2089, 2196, 2031, 1005, 1012, 11830, 11527, 1012, 1001, 14354, 1001, 4105, 1001, 4737, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


En este caso podemos ver cómo el `attention_mask` contiene bastantes `0`, ya que este texto tenía una longitud menor a los 60 tokens que hemos fijado para la tokenización.

Como último paso, necesitamos transformar los campos que entrarán al modelo a tensores de Pytorch, ya que los modelos de Transformers se basan en esta librería de Deep Learning. Esto puede hacerse fácilmente con la siguiente función, a la que le indicamos qué campos queremos transformar.

In [16]:
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Con esto, un dato ha quedado con el siguiente formato

In [17]:
tokenized_dataset["train"][0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
          2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
         14354,  1001,  4105,  1001,  4737,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'label': tensor(2)}

Y podemos confirmar que se trata de tensores de Pytorch

In [18]:
print(type(tokenized_dataset["train"][0]["input_ids"]))
print(type(tokenized_dataset["train"][0]["attention_mask"]))
print(type(tokenized_dataset["train"][0]["label"]))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>


## Modelos de lenguaje

La librería [Transformers de Huggingface](https://huggingface.co/docs/transformers/index) permite utilizar con facilidad diferentes modelos de lenguaje basados en la arquitectura Transformers. Por ejemplo para cargar el modelo DistilBERT ya pre-entrenado podemos usar un `AutoModel`

In [19]:
from transformers import AutoModel

distilbert = AutoModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


La versión pre-entrenada contiene el "cuerpo" del modelo, que puede recibir una secuencia de tokens y devolver embeddings "contextualizados" para cada uno de esos tokens.

<img src="http://jalammar.github.io/images/bert-encoders-input.png">
<div align="right">Image credit: <a href="http://jalammar.github.io/illustrated-bert/">The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)</a></div>

Podemos probar esto con uno de los textos de entrenamiento, utilizando el modelo que hemos cargado como si fuera una función, y pasándole la información necesaria de ese texto: sus `input_ids` y `attention_mask`.

In [20]:
outputs = distilbert(
    input_ids=tokenized_dataset["train"][0:1]["input_ids"],
    attention_mask=tokenized_dataset["train"][0:1]["attention_mask"]
)

El objeto de salidas del modelo que hemos obtenido se comporta como un diccionario que contiene distintos atributos informativos. En el caso del modelo DistilBERT contiene únicamente un campo `last_hidden_state` con los embeddings de salida de la última capa del modelo.

In [21]:
outputs

BaseModelOutput([('last_hidden_state',
                  tensor([[[ 0.0452, -0.1978, -0.1496,  ..., -0.2978,  0.0022,  0.5574],
                           [-0.1270,  0.2253,  0.3226,  ..., -0.1037, -0.3597,  0.3049],
                           [ 0.4912,  0.0353,  0.2810,  ..., -0.0753, -0.4323,  0.3302],
                           ...,
                           [ 0.0684, -0.1066,  0.3897,  ..., -0.0443, -0.4173,  0.1069],
                           [ 0.0308, -0.1521,  0.2782,  ..., -0.0655, -0.5079,  0.2527],
                           [ 0.2016, -0.1871,  0.4356,  ..., -0.0881, -0.3441,  0.0934]]],
                         grad_fn=<NativeLayerNormBackward0>))])

Podemos comprobar que efectivamente hemos obtenido un vector de embedding para cada token que ha entrado al modelo. En el caso de DistilBERT estos embeddings tienen una longitud de 768 valores.

In [22]:
print(f"Input tensor shape {tokenized_dataset['train'][0:1]['input_ids'].shape}")
print(f"DistilBERT embeddings shape {outputs['last_hidden_state'].shape}")

Input tensor shape torch.Size([1, 60])
DistilBERT embeddings shape torch.Size([1, 60, 768])


### Ajuste fino de un modelo de lenguaje

Aunque podríamos pensar en utilizar los embeddings obtenidos como variables explicativas con las que resolver nuestro problema de clasificación, en general esta aproximación no produce buenos resultados, excepto en el caso de aquellos modelos de lenguaje que han sido entrenados explícitamente para producir embeddings con esta finalidad. DistilBERT no es uno de estos modelos, por lo que tendremos que seguir una estrategia diferente.

La estrategia consiste en añadir una "cabeza" de clasificación al modelo, que reciba como entrada el embedding generado para el token especial `[CLS]` y genere la predicción de clase para nuestro problema. Con esta arquitectura de modelo, podemos hacer back-propagation supervisado con nuestro dataset de entrenamiento para obtener el clasificador que nos interesa.

<img src="http://jalammar.github.io/images/bert-classifier.png">
<div align="right">Image credit: <a href="http://jalammar.github.io/illustrated-bert/">The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)</a></div>

La librería Transformers puede preparar esta cambio en la arquitectura. Solo necesitamos volver a cargar el modelo, pero utilizando una versión `AutoModelForSequenceClassification`. Dado que esta arquitectura incluye la cabeza de clasificación, debemos informar del número de clases que tiene nuestro problema, y que conformará el número de salidas de la red neuronal.

In [23]:
from transformers import AutoModelForSequenceClassification
num_labels = len(set(dataset["train"]["label"]))
distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

Ahora que tenemos la arquitectura de red lista, vamos a preparar el entrenamiento. El primer paso consiste en crear un objeto `TrainingArguments`, a través del cual podemos configurar todos los aspectos del proceso de entrenamiento.

In [24]:
from transformers import TrainingArguments

batch_size=16
training_args = TrainingArguments(
    output_dir="test_trainer",  # Carpeta donde se guardará el modelo entrenado
    num_train_epochs=5,  # Número de épocas de entrenamiento
    per_device_train_batch_size=batch_size,  # Tamaño de batch para el entrenamiento
    per_device_eval_batch_size=batch_size,  # Tamaño de batch a la hora de evaluar el rendimiento del modelo
    load_best_model_at_end=True,  # Al final del entrenamiento se recuperará el mejor modelo visto (en error de validación)
    evaluation_strategy="epoch",  # Se medirá el error de validación al final de cada época de entrenamiento
    save_strategy="epoch"  # Se guardará una copia del modelo al final de cada época de entrenamiento
)

También podemos preparar métricas aparte para monitorizar durante el entrenamiento. Por ejemplo, a continuación preparamos una función que calcule la accuracy.

In [25]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

El último paso de preparación es crear un objeto `Trainer`, que será el que realice todo el proceso de entrenamiento.

In [26]:
from transformers import Trainer

trainer = Trainer(
    model=distilbert,  # Modelo de lenguaje sobre el que vamos a entrenar
    args=training_args,  # TrainingArguments que hemos preparado antes
    train_dataset=tokenized_dataset["train"],  # Dataset de entrenamiento
    eval_dataset=tokenized_dataset["validation"],  # Dataset de validación
    compute_metrics=compute_metrics  # Función de cálculo de métricas que hemos preparado
)

Con esto ya podemos entrenar, llamando al método `train`. A medida que se va entrenando el modelo iremos obteniendo información sobre su rendimiento en entrenamiento y validación.

In [27]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3257
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1020


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.597562,0.780749
2,No log,0.624056,0.786096
3,0.537300,0.680427,0.807487
4,0.537300,0.910174,0.794118
5,0.102800,0.927821,0.791444


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 374
  Batch size = 16
Saving model checkpoint to test_trainer/checkpoint-204
Configuration saved in test_trainer/checkpoint-204/config.json
Model weights saved in test_trainer/checkpoint-204/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 374
  Batch size = 16
Saving model checkpoint to test_trainer/checkpoint-408
Configuration saved in test_trainer/checkpoint-408/config.json
Model weights

TrainOutput(global_step=1020, training_loss=0.3151408688694823, metrics={'train_runtime': 237.3398, 'train_samples_per_second': 68.615, 'train_steps_per_second': 4.298, 'total_flos': 252809593293600.0, 'train_loss': 0.3151408688694823, 'epoch': 5.0})

Una vez entrenado el modelo, podemos medir su rendimiento sobre el conjunto de test con el método `evaluate`.

In [28]:
trainer.evaluate(tokenized_dataset["test"])

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1421
  Batch size = 16


{'epoch': 5.0,
 'eval_accuracy': 0.7973258268824771,
 'eval_loss': 0.5737826228141785,
 'eval_runtime': 6.4785,
 'eval_samples_per_second': 219.341,
 'eval_steps_per_second': 13.738}

## Generando predicciones con el modelo

Ahora que tenemos nuestro modelo entrenado, podemos probar a pasarle cualquier texto y comprobar si es capaz de detectar correctamente la emoción que expresa. Para ello, tenemos que construir un objeto `Dataset`, de la siguiente manera.

In [29]:
from datasets import Dataset

custom_dataset = Dataset.from_dict({
    "text": [
        "So hyped for the new Spiderman movie!",
        "I don't understand how you guys keep using twitter. It's terrible piece of trash.",
        "OK, so I watched Grave of the Fireflies. Now I feel miserable.",
        "Our last product launch has been a significant success. We are positive about the future of our company."
    ]
})

De manera equivalente a como preparamos los datos al inicio del notebook, necesitamos realizar la tokenización de estos textos.

In [30]:
tokenized_custom_dataset = custom_dataset.map(
    lambda examples: tokenizer(examples["text"], padding='max_length', max_length=60, truncation=True),
    batched=True,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Con esto ya tenemos los datos listos, y podemos pedir a nuestro modelo que haga predicciones sobre ellos, usando el método `predict`.

In [31]:
preds = trainer.predict(tokenized_custom_dataset)
preds

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 4
  Batch size = 16


PredictionOutput(predictions=array([[-0.57789844,  2.164612  , -1.155689  , -0.8687709 ],
       [ 2.7917821 , -1.4649498 , -1.8211246 , -0.10847265],
       [-0.5569488 , -1.2575734 , -1.6052427 ,  2.4621675 ],
       [-0.94126934,  0.42590326,  0.8077799 , -0.694434  ]],
      dtype=float32), label_ids=None, metrics={'test_runtime': 0.0431, 'test_samples_per_second': 92.743, 'test_steps_per_second': 23.186})

La salida del método `predict` es un objeto `PredictionOutput`, el cual contiene un atributo `predictions` con las probabilidades sin normalizar (logits) de cada clase. Si estamos interesados en obtener la clase más probable, podemos tomar para cada texto el valor más alto de `predictions`. Esto puede hacerse con el método `argmax`.

In [32]:
predicted_classes = preds.predictions.argmax(axis=1)
predicted_classes

array([1, 0, 3, 2])

Para interpretar estos índices de clase, podemos consultar el diccionario `class_labels` que creamos al inicio del notebook, y obtener así los nombres de las clases predichas.

In [33]:
predict_classes_names = [class_labels[c] for c in predicted_classes]
predict_classes_names

['joy', 'anger', 'sadness', 'optimism']

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Prueba a generar predicciones para otros textos creados por ti mismo. ¿Acierta el modelo en su detección de la emoción?
</font>

***

In [34]:
####### INSERT YOUR CODE HERE
custom_dataset = Dataset.from_dict({
    "text": [
        "Go fuck off man",
        "I met Obama at the local burguer! I swear, this is the most epic day in my life!"
    ]
})

tokenized_custom_dataset = custom_dataset.map(
    lambda examples: tokenizer(examples["text"], padding='max_length', max_length=60, truncation=True),
    batched=True,
)

preds = trainer.predict(tokenized_custom_dataset)
predicted_classes = preds.predictions.argmax(axis=1)
predict_classes_names = [class_labels[c] for c in predicted_classes]
predict_classes_names

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2
  Batch size = 16


['anger', 'anger']

## Afrontando otro problema de clasificación

El dataset [tweet_eval](https://huggingface.co/datasets/tweet_eval) que hemos utilizado contiene diversas tareas de clasificación. Para practicar los conceptos anteriores, vamos a repetir el entrenamiento de un modelo con otra de esas tareas: detección de opiniones a favor o en contra de políticas sobre el clima (`stance_climate`)

In [35]:
dataset = load_dataset("tweet_eval", "stance_climate")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/355 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/169 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/stance_climate/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Utiliza DistilBERT para entrenar un modelo sobre este nuevo dataset. Recuerda que los pasos a seguir son:
  <ul>
    <li>Tokenizar el dataset y configurarlo en formato de Pytorch.</li>
    <li>Crear un nuevo AutoModelForSequenceClassification de DistilBERT con el número correcto de clases.</li>
    <li>Preparar los TrainingArguments.</li>
    <li>Preparar el objeto Trainer.</li>
    <li>Entrenar.</li>
    <li>Medir el acierto sobre el conjunto de test.</li>
  </ul>
</font>

***

In [36]:
####### INSERT YOUR CODE HERE
class_labels = {
    0: "neutral",
    1: "against",
    2: "favor",
}

tokenized_dataset = dataset.map(
    lambda example: tokenizer(example["text"], max_length=60, padding='max_length', truncation=True),
    batched=True
)

tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

num_labels = len(set(dataset["train"]["label"]))
distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

batch_size=16
training_args = TrainingArguments(
    output_dir="test_trainer",  # Carpeta donde se guardará el modelo entrenado
    num_train_epochs=5,  # Número de épocas de entrenamiento
    per_device_train_batch_size=batch_size,  # Tamaño de batch para el entrenamiento
    per_device_eval_batch_size=batch_size,  # Tamaño de batch a la hora de evaluar el rendimiento del modelo
    load_best_model_at_end=True,  # Al final del entrenamiento se cargará el mejor modelo visto (en error de validación)
    evaluation_strategy="epoch",  # Se medirá el error de validación al final de cada época de entrenamiento
    save_strategy="epoch"  # Se guardará una copia del modelo al final de cada época de entrenamiento
)

trainer = Trainer(
    model=distilbert,  # Modelo de lenguaje sobre el que vamos a entrenar
    args=training_args,  # TrainingArguments que hemos preparado antes
    train_dataset=tokenized_dataset["train"],  # Dataset de entrenamiento
    eval_dataset=tokenized_dataset["validation"],  # Dataset de validación
    compute_metrics=compute_metrics  # Función de cálculo de métricas que hemos preparado
)

trainer.train()
trainer.evaluate(tokenized_dataset["test"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

loading we

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.716117,0.725000
2,No log,0.461068,0.875000
3,No log,0.458862,0.875000
4,No log,0.494597,0.850000
5,No log,0.435092,0.875000


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
Saving model checkpoint to test_trainer/checkpoint-23
Configuration saved in test_trainer/checkpoint-23/config.json
Model weights saved in test_trainer/checkpoint-23/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 16
Saving model checkpoint to test_trainer/checkpoint-46
Configuration saved in test_trainer/checkpoint-46/config.json
Model weights saved 

{'epoch': 5.0,
 'eval_accuracy': 0.8461538461538461,
 'eval_loss': 0.5211184620857239,
 'eval_runtime': 0.8026,
 'eval_samples_per_second': 210.565,
 'eval_steps_per_second': 13.705}

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Prueba el modelo que acabas de entrenar con algunos textos que tú mismo escribas, a favor y en contra de las políticas de clima. ¿Qué tal funciona tu modelo?
</font>

***

In [37]:
####### INSERT YOUR CODE HERE
custom_dataset = Dataset.from_dict({
    "text": [
        "So much about little Greta, but nobody cares about the actual problems of the people. Climate change is a scam.",
        "It's now or never. There is no planet B!"
    ]
})

tokenized_custom_dataset = custom_dataset.map(
    lambda examples: tokenizer(examples["text"], padding='max_length', max_length=60, truncation=True),
    batched=True,
)

preds = trainer.predict(tokenized_custom_dataset)
predicted_classes = preds.predictions.argmax(axis=1)
predict_classes_names = [class_labels[c] for c in predicted_classes]
predict_classes_names

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2
  Batch size = 16


['favor', 'favor']

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    Intenta mejorar los resultados de tu modelo. Puedes utilizar alguna de las siguientes estrategias:
    <ul>
      <li>Modificar el tamaño de batch</li>
      <li>Modificar la learning rate del aprendizaje</li>
      <li>Utilizar otro modelo de los disponibles en el <href a=https://huggingface.co/>Hub de Huggingface</href></li>
    </ul>
</font>

***

In [38]:
####### INSERT YOUR CODE HERE